In [30]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import os
import torch

In [35]:
df_raw = pd.read_csv(os.path.join('./', 'test.csv'))
df_raw, len(df_raw)

(                   date    HUFL   HULL   MUFL   MULL   LUFL   LULL         OT
 0   2016-07-01 00:00:00   5.827  2.009  1.599  0.462  4.203  1.340  30.531000
 1   2016-07-01 01:00:00   5.693  2.076  1.492  0.426  4.142  1.371  27.787001
 2   2016-07-01 02:00:00   5.157  1.741  1.279  0.355  3.777  1.218  27.787001
 3   2016-07-01 03:00:00   5.090  1.942  1.279  0.391  3.807  1.279  25.044001
 4   2016-07-01 04:00:00   5.358  1.942  1.492  0.462  3.868  1.279  21.948000
 ..                  ...     ...    ...    ...    ...    ...    ...        ...
 95  2016-07-04 23:00:00  12.726  6.497  9.346  3.482  3.168  1.980  25.466000
 96  2016-07-05 00:00:00  11.989  5.626  8.777  2.949  3.198  1.980  25.958000
 97  2016-07-05 01:00:00  12.525  6.296  8.955  3.163  3.137  2.010  25.958000
 98  2016-07-05 02:00:00  12.324  6.296  8.813  3.376  2.985  1.919  26.028000
 99  2016-07-05 03:00:00  10.717  5.425  8.066  2.878  2.833  1.858  28.913000
 
 [100 rows x 8 columns],
 100)

In [122]:
df_raw['OT'][0]-2

28.5310001373291

In [37]:
# cols 只是提取了标签
cols = list(df_raw.columns)
print(cols)
cols.remove('OT'); cols.remove('date')
print(cols)

df_raw = df_raw[['date'] + cols +['OT']]
df_raw.head()

['date', 'HUFL', 'HULL', 'MUFL', 'MULL', 'LUFL', 'LULL', 'OT']
['HUFL', 'HULL', 'MUFL', 'MULL', 'LUFL', 'LULL']


,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
0,2016-07-01 00:00:00,5.827,2.009,1.599,0.462,4.203,1.340,30.531000
1,2016-07-01 01:00:00,5.693,2.076,1.492,0.426,4.142,1.371,27.787001
2,2016-07-01 02:00:00,5.157,1.741,1.279,0.355,3.777,1.218,27.787001
3,2016-07-01 03:00:00,5.090,1.942,1.279,0.391,3.807,1.279,25.044001
4,2016-07-01 04:00:00,5.358,1.942,1.492,0.462,3.868,1.279,21.948000


In [38]:
num_train = int(len(df_raw)*0.7)
num_test = int(len(df_raw)*0.2)
num_vali = len(df_raw) - num_train - num_test
num_train, num_test, num_vali

(70, 20, 10)

In [15]:
seq_len = 30*24           # input sequence length of Informer encoder
label_len = 7*24          # start token length of Informer decoder
pred_len = 24             # prediction sequence length

In [42]:
flag = 0
df_raw_1 = 10000

num_train = int(df_raw_1*0.7)
num_test = int(df_raw_1*0.2)
num_vali = df_raw_1 - num_train - num_test

print(num_train)
print(num_test)
print(num_vali)
print('##########################')

type_map = {'train': 0, 'val': 1, 'test': 2}
set_type = type_map['test']

border1s = [0, num_train-seq_len, df_raw_1-num_test-seq_len]
border2s = [num_train, num_train+num_vali, df_raw_1]
print(border1s)
print(border2s)
print('##########################')

border1 = border1s[set_type]
border2 = border2s[set_type]
print(border1)
print(border2)


7000
2000
1000
##########################
[0, 6280, 7280]
[7000, 8000, 10000]
##########################
7280
10000


In [47]:
features = 'MS'
if features=='M' or features=='MS':
        cols_data = df_raw.columns[1:]
        df_data = df_raw[cols_data]
elif features=='S':
        df_data = df_raw[['OT']]

df_data.head(), len(df_data)

(    HUFL   HULL   MUFL   MULL   LUFL   LULL         OT
 0  5.827  2.009  1.599  0.462  4.203  1.340  30.531000
 1  5.693  2.076  1.492  0.426  4.142  1.371  27.787001
 2  5.157  1.741  1.279  0.355  3.777  1.218  27.787001
 3  5.090  1.942  1.279  0.391  3.807  1.279  25.044001
 4  5.358  1.942  1.492  0.462  3.868  1.279  21.948000,
 100)

In [48]:
data = df_data.values
data

array([[ 5.82700014,  2.00900006,  1.59899998,  0.46200001,  4.20300007,
         1.34000003, 30.53100014],
       [ 5.69299984,  2.07599998,  1.49199998,  0.426     ,  4.1420002 ,
         1.37100005, 27.78700066],
       [ 5.15700006,  1.74100006,  1.27900004,  0.35499999,  3.77699995,
         1.21800005, 27.78700066],
       [ 5.09000015,  1.94200003,  1.27900004,  0.391     ,  3.80699992,
         1.27900004, 25.04400063],
       [ 5.3579998 ,  1.94200003,  1.49199998,  0.46200001,  3.86800003,
         1.27900004, 21.94799995],
       [ 5.62599993,  2.14299989,  1.528     ,  0.53299999,  4.05100012,
         1.37100005, 21.17399979],
       [ 7.16699982,  2.94700003,  2.13199997,  0.78200001,  5.02600002,
         1.85800004, 22.79199982],
       [ 7.43499994,  3.28200006,  2.30999994,  1.03100002,  5.08699989,
         2.22399998, 23.1439991 ],
       [ 5.55900002,  3.01399994,  2.4519999 ,  1.17299998,  2.95499992,
         1.43200004, 21.66699982],
       [ 4.55499983,  2.5450

In [84]:
from typing import List

import numpy as np
import pandas as pd
from pandas.tseries import offsets
from pandas.tseries.frequencies import to_offset

class TimeFeature:
    def __init__(self):
        pass

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        pass

    def __repr__(self):
        return self.__class__.__name__ + "()"

class SecondOfMinute(TimeFeature):
    """Minute of hour encoded as value between [-0.5, 0.5]"""
    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return index.second / 59.0 - 0.5

class MinuteOfHour(TimeFeature):
    """Minute of hour encoded as value between [-0.5, 0.5]"""
    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return index.minute / 59.0 - 0.5

class HourOfDay(TimeFeature):
    """Hour of day encoded as value between [-0.5, 0.5]"""
    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return index.hour / 23.0 - 0.5

class DayOfWeek(TimeFeature):
    """Hour of day encoded as value between [-0.5, 0.5]"""
    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return index.dayofweek / 6.0 - 0.5

class DayOfMonth(TimeFeature):
    """Day of month encoded as value between [-0.5, 0.5]"""
    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return (index.day - 1) / 30.0 - 0.5

class DayOfYear(TimeFeature):
    """Day of year encoded as value between [-0.5, 0.5]"""
    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return (index.dayofyear - 1) / 365.0 - 0.5

class MonthOfYear(TimeFeature):
    """Month of year encoded as value between [-0.5, 0.5]"""
    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return (index.month - 1) / 11.0 - 0.5

class WeekOfYear(TimeFeature):
    """Week of year encoded as value between [-0.5, 0.5]"""
    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return (index.week - 1) / 52.0 - 0.5

def time_features_from_frequency_str(freq_str: str) -> List[TimeFeature]:
    """
    Returns a list of time features that will be appropriate for the given frequency string.
    Parameters
    ----------
    freq_str
        Frequency string of the form [multiple][granularity] such as "12H", "5min", "1D" etc.
    """

    features_by_offsets = {
        offsets.YearEnd: [],
        offsets.QuarterEnd: [MonthOfYear],
        offsets.MonthEnd: [MonthOfYear],
        offsets.Week: [DayOfMonth, WeekOfYear],
        offsets.Day: [DayOfWeek, DayOfMonth, DayOfYear],
        offsets.BusinessDay: [DayOfWeek, DayOfMonth, DayOfYear],
        offsets.Hour: [HourOfDay, DayOfWeek, DayOfMonth, DayOfYear],
        offsets.Minute: [
            MinuteOfHour,
            HourOfDay,
            DayOfWeek,
            DayOfMonth,
            DayOfYear,
        ],
        offsets.Second: [
            SecondOfMinute,
            MinuteOfHour,
            HourOfDay,
            DayOfWeek,
            DayOfMonth,
            DayOfYear,
        ],
    }

    offset = to_offset(freq_str)

    for offset_type, feature_classes in features_by_offsets.items():
        if isinstance(offset, offset_type):
            return [cls() for cls in feature_classes]

    supported_freq_msg = f"""
    Unsupported frequency {freq_str}
    The following frequencies are supported:
        Y   - yearly
            alias: A
        M   - monthly
        W   - weekly
        D   - daily
        B   - business days
        H   - hourly
        T   - minutely
            alias: min
        S   - secondly
    """
    raise RuntimeError(supported_freq_msg)

def time_features(dates, timeenc=1, freq='h'):
    """
    > `time_features` takes in a `dates` dataframe with a 'dates' column and extracts the date down to `freq` where freq can be any of the following if `timeenc` is 0: 
    > * m - [month]
    > * w - [month]
    > * d - [month, day, weekday]
    > * b - [month, day, weekday]
    > * h - [month, day, weekday, hour]
    > * t - [month, day, weekday, hour, *minute]
    > 
    > If `timeenc` is 1, a similar, but different list of `freq` values are supported (all encoded between [-0.5 and 0.5]): 
    > * Q - [month]
    > * M - [month]
    > * W - [Day of month, week of year]
    > * D - [Day of week, day of month, day of year]
    > * B - [Day of week, day of month, day of year]
    > * H - [Hour of day, day of week, day of month, day of year]
    > * T - [Minute of hour*, hour of day, day of week, day of month, day of year]
    > * S - [Second of minute, minute of hour, hour of day, day of week, day of month, day of year]

    *minute returns a number from 0-3 corresponding to the 15 minute period it falls into.
    """
    if timeenc==0:
        dates['month'] = dates.date.apply(lambda row:row.month,1)
        dates['day'] = dates.date.apply(lambda row:row.day,1)
        dates['weekday'] = dates.date.apply(lambda row:row.weekday(),1)
        dates['hour'] = dates.date.apply(lambda row:row.hour,1)
        dates['minute'] = dates.date.apply(lambda row:row.minute,1)
        dates['minute'] = dates.minute.map(lambda x:x//15)
        freq_map = {
            'y':[],'m':['month'],'w':['month'],'d':['month','day','weekday'],
            'b':['month','day','weekday'],'h':['month','day','weekday','hour'],
            't':['month','day','weekday','hour','minute'],
        }
        return dates[freq_map[freq.lower()]].values
    if timeenc==1:
        dates = pd.to_datetime(dates.date.values)
        return np.vstack([feat(dates) for feat in time_features_from_frequency_str(freq)]).transpose(1,0)


In [111]:

b = 10
df_stamp = df_raw[['date']][b:70]
# pri = df_stamp['date']
# print(f'data_stamp date from {df_stamp.date[0]} to {pri[len(pri)-1]} with len {len(pri)}')

# df_stamp, df_stamp.shape, df_stamp['date'][0], df_stamp['date'][len(df_stamp)-1]



<bound method IndexOpsMixin.argmin of 10    2016-07-01 10:00:00
11    2016-07-01 11:00:00
12    2016-07-01 12:00:00
13    2016-07-01 13:00:00
14    2016-07-01 14:00:00
15    2016-07-01 15:00:00
16    2016-07-01 16:00:00
17    2016-07-01 17:00:00
18    2016-07-01 18:00:00
19    2016-07-01 19:00:00
20    2016-07-01 20:00:00
21    2016-07-01 21:00:00
22    2016-07-01 22:00:00
23    2016-07-01 23:00:00
24    2016-07-02 00:00:00
25    2016-07-02 01:00:00
26    2016-07-02 02:00:00
27    2016-07-02 03:00:00
28    2016-07-02 04:00:00
29    2016-07-02 05:00:00
30    2016-07-02 06:00:00
31    2016-07-02 07:00:00
32    2016-07-02 08:00:00
33    2016-07-02 09:00:00
34    2016-07-02 10:00:00
35    2016-07-02 11:00:00
36    2016-07-02 12:00:00
37    2016-07-02 13:00:00
38    2016-07-02 14:00:00
39    2016-07-02 15:00:00
40    2016-07-02 16:00:00
41    2016-07-02 17:00:00
42    2016-07-02 18:00:00
43    2016-07-02 19:00:00
44    2016-07-02 20:00:00
45    2016-07-02 21:00:00
46    2016-07-02 22:00:00


In [115]:
# pd.to_datetime() 是 Pandas 提供的一个方法，用于将输入的数据（通常是字符串或数值形式的日期）转换为日期时间格式。
df_stamp['date'] = pd.to_datetime(df_stamp.date)
df_stamp, type(df_stamp['date']), df_stamp.shape

(                  date
 10 2016-07-01 10:00:00
 11 2016-07-01 11:00:00
 12 2016-07-01 12:00:00
 13 2016-07-01 13:00:00
 14 2016-07-01 14:00:00
 15 2016-07-01 15:00:00
 16 2016-07-01 16:00:00
 17 2016-07-01 17:00:00
 18 2016-07-01 18:00:00
 19 2016-07-01 19:00:00
 20 2016-07-01 20:00:00
 21 2016-07-01 21:00:00
 22 2016-07-01 22:00:00
 23 2016-07-01 23:00:00
 24 2016-07-02 00:00:00
 25 2016-07-02 01:00:00
 26 2016-07-02 02:00:00
 27 2016-07-02 03:00:00
 28 2016-07-02 04:00:00
 29 2016-07-02 05:00:00
 30 2016-07-02 06:00:00
 31 2016-07-02 07:00:00
 32 2016-07-02 08:00:00
 33 2016-07-02 09:00:00
 34 2016-07-02 10:00:00
 35 2016-07-02 11:00:00
 36 2016-07-02 12:00:00
 37 2016-07-02 13:00:00
 38 2016-07-02 14:00:00
 39 2016-07-02 15:00:00
 40 2016-07-02 16:00:00
 41 2016-07-02 17:00:00
 42 2016-07-02 18:00:00
 43 2016-07-02 19:00:00
 44 2016-07-02 20:00:00
 45 2016-07-02 21:00:00
 46 2016-07-02 22:00:00
 47 2016-07-02 23:00:00
 48 2016-07-03 00:00:00
 49 2016-07-03 01:00:00
 50 2016-07-03 0

In [96]:
pred_dates = pd.date_range(df_stamp.date.values[-1], periods=24+1, freq='h')
pred_dates, len(pred_dates)

(DatetimeIndex(['2016-07-03 21:00:00', '2016-07-03 22:00:00',
                '2016-07-03 23:00:00', '2016-07-04 00:00:00',
                '2016-07-04 01:00:00', '2016-07-04 02:00:00',
                '2016-07-04 03:00:00', '2016-07-04 04:00:00',
                '2016-07-04 05:00:00', '2016-07-04 06:00:00',
                '2016-07-04 07:00:00', '2016-07-04 08:00:00',
                '2016-07-04 09:00:00', '2016-07-04 10:00:00',
                '2016-07-04 11:00:00', '2016-07-04 12:00:00',
                '2016-07-04 13:00:00', '2016-07-04 14:00:00',
                '2016-07-04 15:00:00', '2016-07-04 16:00:00',
                '2016-07-04 17:00:00', '2016-07-04 18:00:00',
                '2016-07-04 19:00:00', '2016-07-04 20:00:00',
                '2016-07-04 21:00:00'],
               dtype='datetime64[ns]', freq='H'),
 25)

In [93]:
data_stamp = time_features(df_stamp, timeenc=1, freq='h')
data_stamp, data_stamp.shape

(array([[-0.5       ,  0.16666667, -0.5       , -0.00136986],
        [-0.45652174,  0.16666667, -0.5       , -0.00136986],
        [-0.41304348,  0.16666667, -0.5       , -0.00136986],
        [-0.36956522,  0.16666667, -0.5       , -0.00136986],
        [-0.32608696,  0.16666667, -0.5       , -0.00136986],
        [-0.2826087 ,  0.16666667, -0.5       , -0.00136986],
        [-0.23913043,  0.16666667, -0.5       , -0.00136986],
        [-0.19565217,  0.16666667, -0.5       , -0.00136986],
        [-0.15217391,  0.16666667, -0.5       , -0.00136986],
        [-0.10869565,  0.16666667, -0.5       , -0.00136986],
        [-0.06521739,  0.16666667, -0.5       , -0.00136986],
        [-0.02173913,  0.16666667, -0.5       , -0.00136986],
        [ 0.02173913,  0.16666667, -0.5       , -0.00136986],
        [ 0.06521739,  0.16666667, -0.5       , -0.00136986],
        [ 0.10869565,  0.16666667, -0.5       , -0.00136986],
        [ 0.15217391,  0.16666667, -0.5       , -0.00136986],
        

In [112]:
df_stamp_test = pd.DataFrame(columns = ['date'])
df_stamp_test

,date


In [125]:
num = 11
num % 10 == 0

False

In [126]:
import time
time.sleep(5)